![alt text](../../pythonexposed-high-resolution-logo-black.jpg "Optionele titel")

### Named Tuples - DocStrings en Default Values

In [4]:
from collections import namedtuple

#### DocStrings toevoegen aan named tuples

Dit is gemakkelijk te doen, zowel met de gegenereerde class als met zijn eigenschappen.

In [5]:
Point2D = namedtuple('Point2D', 'x y')

In [6]:
Point2D.__doc__ = 'Vertegenwoordigt een 2D-cartesiaanse coördinaat'

En we kunnen zelfs docstrings toevoegen aan de eigenschappen:

In [7]:
Point2D.x.__doc__ = 'x-coordinaat'
Point2D.y.__doc__ = 'y-coordinaat'

In [8]:
help(Point2D)

Help on class Point2D in module __main__:

class Point2D(builtins.tuple)
 |  Point2D(x, y)
 |  
 |  Vertegenwoordigt een 2D-cartesiaanse coördinaat
 |  
 |  Method resolution order:
 |      Point2D
 |      builtins.tuple
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getnewargs__(self)
 |      Return self as a plain tuple.  Used by copy and pickle.
 |  
 |  __repr__(self)
 |      Return a nicely formatted representation string
 |  
 |  _asdict(self)
 |      Return a new dict which maps field names to their values.
 |  
 |  _replace(self, /, **kwds)
 |      Return a new Point2D object replacing specified fields with new values
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  _make(iterable) from builtins.type
 |      Make a new Point2D object from a sequence or iterable
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__

#### Default values toevoegen aan named tuples

#### Het gebruiken van een Prototype

Deze techniek staat in de Python-documentatie en maakt gebruik van het concept van het creëren van een prototype object waarbij de standaardwaarden zijn ingesteld:

In [11]:
Vector = namedtuple('Vector', 'x1 y1 x2 y2 origin_x origin_y')

In [12]:
vector_zeroorigin = Vector(x1=None, y1=None, x2=None, y2=None, origin_x=0, origin_y=0)

In [13]:
vector_zeroorigin

Vector(x1=None, y1=None, x2=None, y2=None, origin_x=0, origin_y=0)

De genoemde tuple `vector_zeroorigin` is nu een prototype van een vector met nul oorsprong.
Om nieuwe vectoren te creëren met die oorsprong als standaard, gebruiken we niet langer de Vector klasse, maar in plaats daarvan gebruiken we _replace als volgt:

In [14]:
v1 = vector_zeroorigin._replace(x1=1, y1=1, x2=10, y2=10)

In [15]:
v1

Vector(x1=1, y1=1, x2=10, y2=10, origin_x=0, origin_y=0)

Dit werkt zeker en kan nuttig zijn in gevallen waar je meer dan één prototype wilt (bijv. `vector_zeroorigin` en `vector_otherorigin`)

#### Gebruik van `__defaults__`

Er is een alternatieve manier om dit te doen. En, naar mijn mening, een veel schoner alternatief.

In Python worden de standaardwaarden voor de parameters van een functie opgeslagen als een tuple in het attribuut `__defaults__`.


In [11]:
def func(a, b=20, c=30):
    print(a, b, c)

In [12]:
func.__defaults__

(20, 30)

In [13]:
func(10)

10 20 30


Maar de `__defaults__` eigenschap is beschrijfbaar:

In [14]:
func.__defaults__ = (200, 300)

In [15]:
func(10)

10 200 300


In dit geval is de functie waarin we geïnteresseerd zijn om standaardwaarden te specificeren, de named tuple class constructor, d.w.z. `__new__`.
Dus, we hoeven alleen maar `Vector.__new__.__defaults__` in te stellen op de gewenste tuple van standaardwaarden.

Het enige dat opgemerkt moet worden is dat als je minder standaardwaarden opgeeft (bijvoorbeeld m waarden) dan het totale aantal argumenten (bijvoorbeeld n waarden, waar m < n), dan zullen de standaarden van toepassing zijn op de laatste m waarden. Denk erover alsof je je veldnamen en standaardwaarden op twee regels schrijft en deze rechts uitlijnt. (Als je er meer opgeeft, dan worden de waarden aan het begin effectief genegeerd)

In [16]:
Vector.__new__.__defaults__ = (0, 0)

Hier stel ik in feite standaardwaarden in voor enkel de laatste twee elementen, namelijk `origin_x` en `origin_y`.

In [17]:
v1 = Vector(0, 0, 10, 10, -10, -10)

In [18]:
v1

Vector(x1=0, y1=0, x2=10, y2=10, origin_x=-10, origin_y=-10)

In [19]:
v2 = Vector(5, 5, 20, 20)

In [20]:
v2

Vector(x1=5, y1=5, x2=20, y2=20, origin_x=0, origin_y=0)

In [21]:
v3 = Vector(x1=1, y1=1, x2=10, y2=10)

In [22]:
v3

Vector(x1=1, y1=1, x2=10, y2=10, origin_x=0, origin_y=0)

Een nog eenvoudigere manier om standaardwaarden in te stellen als je wilt dat **alle** standaardwaarden hetzelfde zijn:

In [23]:
Vector.__new__.__defaults__ = (0,) * len(Vector._fields)

In [24]:
v5 = Vector()

In [25]:
v5

Vector(x1=0, y1=0, x2=0, y2=0, origin_x=0, origin_y=0)

Uiteraard geldt hier ook de gebruikelijke waarschuwing om geen mutabele standaardwaarden te gebruiken.